In [1]:
import time
import datetime as dt
import sys, os
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Hospital Patient-Flow Modeling

__Simulation Wishlist:__
- Incorporate holidays
- Set capacity limits for non-OR/ER nodes.
- Have ping-pong effects (PAT back to origin, post-op back to OR, post-op back to pre-op, long-term care back to scheduled)
- Schedule micro-delays in the OR after patient moves from the OR to POST-OP nodes to represent OR clean-up
- [OPTIONAL] Have patients that arrive at the queue first, get treated first (SEE NOTES).

__Simulation ReadMe:__

- Modelling Patient Arrival Times
    - The interval between patient arrivals is estimated based on a user input for the number of patients processed per year. 
        - _Running Example: UIHC's Main OR for FY2019 = 19490. This indicates a rate of 1 patient every 0.45 hours._
    - Patient types are broken down into SCHEDULED, FLOOR, or ED TRAUMA by frequency of occurence.
        - _Let's say UIHC is 50% SCHEDULED, 30% FLOOR, and 20% ED TRAUMA._
    - Non-emergency (SCHEDULED and FLOOR) patients are assumed to __arrive__ during business hours (9AM-5PM). There are 2,088 business hours in 2019. This is used to calculate the expected hour intervals between arrivals.
        - $$\frac{2088}{0.8 \times 19490} = 0.13$$
    - Emergency (ED TRAUMA) patients arrive on a 24/7 basis.
        - $$\frac{365 \times 24}{0.2 \times 19490} = 2.25$$
    - The actual arrival intervals are then randomly sampled from the exponential distribution where the scale parameter is set to the expected intervals calculated above.

- Modelling Pre-processing Stages
    - Unlike the ORs, there's no capacity limit for PAT, REGISTERED, PRE-OP, and QUEUE nodes.
    - Due to randomization, patients will travel through these nodes at different rates, so overtaking is possible.
     
- Modelling OR and ER Queues and Operations
     - Once they've arrived at the queue, patients are treated __strictly__ in the order of their arrival at the hospital.
         - _E.g. There are 12 ORs. Patient $i$ is the $i^{th}$ patient to arrive. Regardless of how long they take to move through the pre-processing stages, patients 1-12 will __not have to queue__ for ORs 1-12. Let's assume patient 13 moves quickly through pre-processing, arriving at the OR QUEUE __before__ patients 1-12. She __still has to queue__, allowing patients 1-12 overtake her into the ORs. Patient 13 takes the next OR that opens up._
     - There is at most 1 patient per OR / ER at any one time.
     - If there's a queue, the end time of an operation is the start time of the next patient's operation. No lag time accounting for hospital turnover is included.
- Modelling Post-processing Stages
    - Unlike the ORs, there's no capacity limit for POST-OP nodes.
    - The patient's mode of discharge (DISCHARGED, LONG-TERM CARE, or FLOOR), is selected from a matrix of probabilities depending on whether they were SCHEDULED, FLOOR, or ED TRAUMA.

# Reformatting

## Breakdown of Surgical Specialties
Operating times are loosely defined from the following link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5609617/pdf/1679-4508-eins-15-02-0200.pdf

In [2]:
specialty_dict = {'specialty': ['Anesthesia', 'Dentistry', 'Dermatology', 'Gynecology', 'Int Med - BMT',
                                'Neurology', 'Neurosurgery', 'Ophthalmology', 'Orthopaedics', 'Otolaryngology',
                                'Radiology', 'Surgery', 'Urology'],
                  'operating_time': [2.969, 2, 3, 2.141, 2,
                                     0.6, 3.64, 2.867, 3.8, 3.248,
                                     2, 3.63, 2.567
                                     ],
                  'frequency': [0.0009, 0.0316, 0.0007, 0.0679, 0.0008,
                                0.0145, 0.0914, 0.1182, 0.252, 0.1075,
                                0.0042, 0.2515, 0.0588]
                  }
df_specialty = pd.DataFrame(specialty_dict).set_index('specialty')
df_specialty

,operating_time,frequency
specialty,,
Anesthesia,2.969,0.0009
Dentistry,2.000,0.0316
Dermatology,3.000,0.0007
Gynecology,2.141,0.0679
Int Med - BMT,2.000,0.0008
Neurology,0.600,0.0145
Neurosurgery,3.640,0.0914
Ophthalmology,2.867,0.1182
Orthopaedics,3.800,0.2520


In [3]:
def specialty_generator(df):
    """
    Function to generate the medical field that the patient's operation is categorized in,
    and the corresponding average operating time, based on an input dataframe (df_specialty).
    """
    sp_type = np.random.choice(a=df_specialty.index,
                               p=df_specialty.frequency)
    or_duration = df.loc[sp_type].operating_time
    return sp_type, or_duration

## Breakdown of Appointment Type and Outcome 

In [4]:
pt_dict = {'appointment': ['SCHEDULED', 'ED TRAUMA', 'FLOOR'],
           'frequency': [0.5, 0.2, 0.3],
           'DISCHARGED': [0.5, 0.1, 0.2],
           'LONG-TERM CARE': [0.4, 0.3, 0.6],
           'FLOOR': [0.1, 0.6, 0.2]
           }
df_pt = pd.DataFrame(pt_dict).set_index('appointment')
df_pt

,frequency,DISCHARGED,LONG-TERM CARE,FLOOR
appointment,,,,
SCHEDULED,0.5,0.5,0.4,0.1
ED TRAUMA,0.2,0.1,0.3,0.6
FLOOR,0.3,0.2,0.6,0.2


In [5]:
def pt_generator(df):
    """
    Function to generate a patient's manner of appointment for the operation,
    and the mode of their discharge, based on an input dataframe (df_pt).
    """
    pt_type = np.random.choice(a=df.index, p=df.frequency)
    dc_type = np.random.choice(a=df.loc[pt_type][1:].index, p=df.loc[pt_type][1:])
    return pt_type, dc_type

## Timestamp Generator

In [6]:
def ts_generator(time_list):
    """
    Function to generate sequential timestamps based on cumulatively summing samples from an exponential
    distribution, based on an input list of expected average wait times between events.
    """
    deltas = [time_list[0]]
    for i in time_list[1:]:
        gen = np.random.exponential(i)
        deltas.append(gen)
    return list(np.cumsum(deltas))

## Building Patient Class

In [7]:
class Patient:
    """
    Given inputs, constructs a profile of a patient that is arrives at a hospital. 
    
    Attributes
    ----------
    pt_number : int 
        Patient number - a count that increments everytime a patient profile is instantiated.
        This also marks the order in which patient arrivals occur.
    id : int
        ID unique to each `Patient` instance.
    pt_type : str
        The patient's appointment type (SCHEDULED, ED TRAUMA, or from hospital FLOORs)
    dc_type : str
        How the patient is discharged (DISCHARGED, LONG-TERM CARE, or back to hospital FLOORs)
    sp_type : str
        The medical specialty that the surgery falls under.
    or_time : float
        The expected time taken to complete surgery given the sp_type.
    ed_start_ts : float
        Denotes the arrival time of patients who need the emergency room.
        Only present for ED TRAUMA patients, who can arrive at any time of the day. 
    start_ts : float
        Denotes the arrival time of non-emergency patient cases. Only present for 
        SCHEDULED and FLOOR patients, who only arrive during business hours (9AM-5PM).
    """
    pt_number = 0
    start_ts = pd.Timestamp(dt.datetime(2019, 1, 1)).timestamp()
    ed_start_ts = pd.Timestamp(dt.datetime(2019, 1, 1)).timestamp()

    def __init__(self, pt_df, specialty_df, num_pt_per_year=19490):
        """
        Parameters
        ----------
        pt_df : Pandas dataframe
            Matrix of probabilities denoting the likelihood of occurrence of each patient type,
            and their corresponding mode of discharge.
        specialty_df : Pandas dataframe
            Dataframe describing the likelihood that a surgery relates to a medical specialty,
            and the corresponding expected surgery time in that specialty.
        num_pt_per_year : int
            Number of patients that the practice processes every year, defaults to 19,490.
        """
        # Increment patient counter and assign to instance attribute
        Patient.pt_number += 1
        self.pt_number = Patient.pt_number

        # Generate unique patient ID
        self.id = id(self)

        # Generate where patient arrives from, and mode of discharge
        self.pt_type, self.dc_type = pt_generator(pt_df)

        # Generate medical specialty, and corresponding average operating time
        self.sp_type, self.or_time = specialty_generator(specialty_df)

        # Estimate average time interval between patient arrivals
        freq_series = pt_df.frequency * num_pt_per_year
        ed_trauma_interval = (365*24) / \
            freq_series['ED TRAUMA']  # ER operates 24/7
        scheduled_floor_interval = 2088 / \
            freq_series[['SCHEDULED', 'FLOOR']].sum()  # Business hours only

        # Generate start time for patient arrival drawn from exponential distribution
        if self.pt_type == 'ED TRAUMA':
            Patient.ed_start_ts += np.random.exponential(dt.timedelta(hours=ed_trauma_interval)
                                                         .total_seconds())
            self.ed_start_ts = Patient.ed_start_ts
        else:
            Patient.start_ts += np.random.exponential(dt.timedelta(hours=scheduled_floor_interval)
                                                      .total_seconds())
            # Use an offset to adjust timestamp to valid business hours if needed
            offset = pd.offsets.BusinessHour()
            Patient.start_ts = offset.rollforward(
                pd.Timestamp.utcfromtimestamp(Patient.start_ts)).timestamp()
            self.start_ts = Patient.start_ts

## Building Simulation Class

In [8]:
class Simulation:
    """
    Given a `Patient` instance, generates an event log of that patient's path through
    the hospital floor.
    
    Attributes
    ----------
    pt : `Patient` instance object 
        An instance of the `Patient` class representing a patient profile.
    or_arrival_time : float
        Denotes the arrival time of SCHEDULED and FLOOR patients to the OR QUEUE. With each new instance,
        corresponding class attribute is updated with the latest arrival.
    er_arrival_time : float
        Denotes the arrival time of ED TRAUMA patients to the ER QUEUE. With each new instance,
        corresponding class attribute is updated with the latest arrival.
    hosp_dict : dict
        Dictionary to populate the patient's event log with.
    """
    or_arrival_time = {'OR 1': 0.0, 'OR 2': 0.0, 'OR 3': 0.0, 'OR 4': 0.0,
                       'OR 5': 0.0, 'OR 6': 0.0, 'OR 7': 0.0, 'OR 8': 0.0,
                       'OR 9': 0.0, 'OR 10': 0.0, 'OR 11': 0.0, 'OR 12': 0.0, }
    er_arrival_time = {'ER 1': 0.0, 'ER 2': 0.0, 'ER 3': 0.0}

    def __init__(self, pt_instance):
        """
        Parameters
        ----------
        pt_instance : `Patient` instance object
            An instance of the `Patient` class representing a patient profile.
        """
        self.pt = pt_instance
        self.or_arrival_time = Simulation.or_arrival_time
        self.er_arrival_time = Simulation.er_arrival_time
        self.hosp_dict = {'ticket': [], 'patient_id': [],
                          'node': [], 'specialty': [], 'ts': []}

    def generate_data(self):
        """
        Please see help(Simulation) for more info
        """
        # Outpatient surgery scheduled ahead of time, or inpatient surgery from 'floors'
        if (self.pt.pt_type == 'SCHEDULED') or (self.pt.pt_type == 'FLOOR'):

            # Increment the patient count
            self.hosp_dict['ticket'].extend(8*[self.pt.pt_number])

            # Extend Patient ID
            self.hosp_dict['patient_id'].extend(8*[self.pt.id])

            # Have patients go to the next available OR
            min_wait = min(self.or_arrival_time, key=self.or_arrival_time.get)

            # Extend nodes with patient's path through hospital floor
            self.hosp_dict['node'].extend([self.pt.pt_type, 'PAT', 'REGISTERED', 'PRE-OP', 'OR QUEUE',
                                           min_wait, 'POST-OP', self.pt.dc_type
                                           ])
            # Extend medical specialty type
            self.hosp_dict['specialty'].extend(8*[self.pt.sp_type])

            # Generate and extend timestamps until queueing stage
            self.intervals_1 = ts_generator(time_list=[self.pt.start_ts,
                                                       dt.timedelta(
                                                           hours=1).total_seconds(),
                                                       dt.timedelta(
                                                           hours=1).total_seconds(),
                                                       dt.timedelta(
                                                           hours=0.5).total_seconds(),
                                                       dt.timedelta(
                                                           hours=1).total_seconds()
                                                       ])
            # Check whether queueing is required extend timestamps until end
            if self.intervals_1[-1] >= self.or_arrival_time[min_wait]:
                self.intervals_2 = ts_generator(time_list=[self.intervals_1[-1],
                                                           dt.timedelta(
                                                               hours=self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=1.5).total_seconds()
                                                           ])
                Simulation.or_arrival_time[min_wait] = self.intervals_2[1]
            else:
                self.intervals_2 = ts_generator(time_list=[self.or_arrival_time[min_wait],
                                                           dt.timedelta(
                                                               hours=self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=1.5).total_seconds()
                                                           ])
                Simulation.or_arrival_time[min_wait] = self.intervals_2[1]
            self.hosp_dict['ts'].extend(self.intervals_1 + self.intervals_2)

        # Unscheduled patient requires emergency treatment
        elif self.pt.pt_type == 'ED TRAUMA':
            self.hosp_dict['ticket'].extend(5*[self.pt.pt_number])
            self.hosp_dict['patient_id'].extend(5*[self.pt.id])
            min_wait = min(self.er_arrival_time, key=self.er_arrival_time.get)
            self.hosp_dict['node'].extend(
                ['ED TRAUMA', 'ER QUEUE', min_wait, 'POST-OP', self.pt.dc_type])
            self.hosp_dict['specialty'].extend(5*[self.pt.sp_type])
            self.intervals_1 = ts_generator(time_list=[self.pt.ed_start_ts,
                                                       dt.timedelta(
                                                           hours=0.25).total_seconds()
                                                       ])
            if self.intervals_1[-1] >= self.er_arrival_time[min_wait]:
                self.intervals_2 = ts_generator(time_list=[self.intervals_1[-1],
                                                           dt.timedelta(
                                                               hours=1.5*self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=2.5).total_seconds()
                                                           ])
                Simulation.er_arrival_time[min_wait] = self.intervals_2[1]
            else:
                self.intervals_2 = ts_generator(time_list=[self.er_arrival_time[min_wait],
                                                           dt.timedelta(
                                                               hours=1.5*self.pt.or_time).total_seconds(),
                                                           dt.timedelta(
                                                               hours=2.5).total_seconds()
                                                           ])
                Simulation.er_arrival_time[min_wait] = self.intervals_2[1]
            self.hosp_dict['ts'].extend(self.intervals_1 + self.intervals_2)

# Simulation 

In [9]:
start = time.time()

df_sim = {'ticket': [], 'patient_id': [],
          'node': [], 'specialty': [], 'ts': []}
for i in range(1000):

    # Call Patient class
    pt = Patient(num_pt_per_year=19490, pt_df=df_pt, specialty_df=df_specialty)
    sim = Simulation(pt)
    sim.generate_data()

    # Append to simulation dictionary
    df_sim['ticket'].extend(sim.hosp_dict['ticket'])
    df_sim['patient_id'].extend(sim.hosp_dict['patient_id'])
    df_sim['node'].extend(sim.hosp_dict['node'])
    df_sim['specialty'].extend(sim.hosp_dict['specialty'])
    df_sim['ts'].extend(sim.hosp_dict['ts'])

end = time.time()
print(end-start, 'runtime')

5.10634708404541 runtime


In [10]:
df_sim = pd.DataFrame(data=df_sim)
df_sim.patient_id = df_sim.patient_id.astype(int)
df_sim.ticket = df_sim.ticket.astype(int)

In [11]:
df_clean = df_sim.copy()
df_clean.ts = df_sim.ts.apply(lambda x: pd.Timestamp.utcfromtimestamp(x))
df_clean.head()

,ticket,patient_id,node,specialty,ts
0,1,4449269744,FLOOR,Orthopaedics,2019-01-01 09:00:00.000000
1,1,4449269744,PAT,Orthopaedics,2019-01-01 09:39:20.313152
2,1,4449269744,REGISTERED,Orthopaedics,2019-01-01 11:20:09.438295
3,1,4449269744,PRE-OP,Orthopaedics,2019-01-01 11:28:57.522175
4,1,4449269744,OR QUEUE,Orthopaedics,2019-01-01 12:19:06.093609


# EDA

In [12]:
df_sort = df_clean.sort_values(by='ts', kind='mergesort').reset_index(drop=True).drop(columns='ticket')
df_sort.to_csv('hospital_sim.csv')
df_sort.head()

,patient_id,node,specialty,ts
0,4449269968,ED TRAUMA,Surgery,2019-01-01 00:08:55.928110
1,4449269968,ER QUEUE,Surgery,2019-01-01 00:24:43.386417
2,4449269968,ER 1,Surgery,2019-01-01 00:24:43.386417
3,4449269968,POST-OP,Surgery,2019-01-01 00:59:52.157369
4,4449271312,ED TRAUMA,Surgery,2019-01-01 01:24:18.126014


In [13]:
df_sort['node_delta'] = df_sort.groupby('node').ts.diff().dt.total_seconds()/60
df_sort.groupby('node').node_delta.describe()

,count,mean,std,min,25%,50%,75%,max
node,,,,,,,,
DISCHARGED,307.0,82.499044,209.886009,0.111699,16.494427,42.141086,94.068421,2927.624066
ED TRAUMA,201.0,128.347467,129.898964,1.207723,38.095639,82.611381,177.393575,661.402877
ER 1,65.0,377.273543,292.396506,18.171534,179.317982,313.712600,476.911690,1591.946759
ER 2,71.0,360.113794,339.831542,3.063791,125.947915,259.771671,483.190167,1696.472284
ER 3,63.0,404.975955,400.028381,5.877131,88.360545,297.992509,583.184054,1987.265836
ER QUEUE,201.0,128.316748,128.767238,0.629363,43.121849,84.659892,159.998916,671.884035
FLOOR,563.0,45.809299,107.718554,0.020980,6.905027,17.115377,44.847106,2040.254542
LONG-TERM CARE,443.0,56.687854,90.769244,0.047042,11.786324,28.985512,63.319220,822.438351
OR 1,62.0,398.117739,735.154394,0.489625,75.039582,141.105761,402.814817,4064.006602
